# **추출 체인(Extraction Chain) 구축하기**

이 노트북에서는 **채팅 모델(Chat Models)**의 **도구 호출(Tool Calling)** 기능을 사용하여 **비정형 텍스트에서 구조화된 정보를 추출**하는 방법을 다룹니다. 또한 이 맥락에서 **Few-Shot 프롬프팅(Few-Shot Prompting)**을 사용하여 성능을 개선하는 방법을 시연할 것입니다.

## **스키마 (The Schema)**  

먼저, 텍스트에서 어떤 정보를 추출할 것인지 **정의**해야 합니다.  

이를 위해 **Pydantic**을 사용하여 **개인 정보(personal information)**를 추출하기 위한 예제 **스키마(schema)**를 정의할 것입니다.

In [ ]:
# !pip install -qU \
# python-dotenv \
# langchain \
# langchain-community \
# openai \
# anthropic \
# langchain-openai \
# langchain-anthropic \
# langchain-google-genai \
# python-dotenv

In [ ]:
class Person(BaseModel):
    # 이 Doc-string은 LLM에 'Person' 스키마에 대한 설명으로 전달됩니다.
    # 사람의 이름
    # 사람의 머리 색상
    # 사람의 키 (미터 단위)

## **스키마 정의의 두 가지 사례**

1. **속성(attributes)**과 **스키마(schema)** 를 Pydantic으로 문서화:  
   - 이 정보는 LLM에 전달되며, Pydantic 스키마를 통해 명확하게 정의되어 정보 추출의 품질을 개선하는 데 사용됩니다.
     
<pr></pr>

2. **LLM이 정보를 지어내지 않도록 하세요!**  
   - 위에서 각 속성에 `Optional`을 사용하여 LLM이 답을 모를 경우 `None`을 반환할 수 있도록 했습니다.
  
최상의 성능을 얻으려면 **스키마를 잘 문서화**하고, 텍스트에 추출할 정보가 없을 경우 모델이 결과를 **강제로 반환하지 않도록** 설정합니다.  


## **추출기 (The Extractor)**

이제 위에서 정의한 **스키마(schema)**를 사용하여 **정보 추출기(Information Extractor)**를 만들어 봅니다.

In [ ]:
# 사용자 정의 프롬프트 템플릿 정의
# 텍스트에서 정보를 추출하기 위한 명확한 지침과 추가 컨텍스트를 제공합니다.

**기능/도구 호출(Function/Tool Calling)**을 지원하는 모델을 사용해야 합니다.

In [ ]:
# LLM에서 구조화된 출력을 생성하도록 스키마 바인딩

LLM은 생성 모델이므로, 센티미터로 제공된 신장의 정보를 미터로 정확하게 추출하는 등의 놀라운 작업을 수행할 수 있습니다! 또한 스키마에 정의된 이름, 머리색, 키 외의 다른 내용은 무시하고 답변을 생성합니다.


## 다중 엔터티

많은 경우, 단일 엔티티가 아닌 여러 엔티티를 추출해야 합니다. 이는 Pydantic에서 모델을 서로 중첩하여 쉽게 구현할 수 있습니다.

In [ ]:
class Data(BaseModel):

**여러 엔티티**를 추출할 수 있도록 스키마가 설계되면, 텍스트에 관련 정보가 없을 경우 **빈 리스트(empty list)**를 반환하여 **아무런 엔티티도 추출하지 않을 수 있습니다.**  

이는 일반적으로 **좋은 설계**입니다! 이를 통해 모델이 해당 엔티티를 반드시 감지하도록 강제하지 않을 수 있습니다.  

## **Few-Shot Prompting**  

LLM 애플리케이션의 동작은 **Few-Shot 프롬프팅**을 사용하여 조정할 수 있습니다.  

**챗 모델(Chat Models)**의 경우, 원하는 동작을 보여주는 **입력(input)**과 **응답(response)** 메시지 쌍의 시퀀스로 구성될 수 있습니다.  

예를 들어, `user`와 `assistant` **메시지**가 번갈아 가며 나타나는 구조를 통해 `🦜` 기호의 의미를 전달할 수 있습니다.

**구조화된 출력(Structured Output)**은 종종 도구 호출(Tool Calling)을 기반으로 작동합니다. 이는 일반적으로 **도구 호출을 포함하는 AI 메시지(AI messages)**와 **도구 호출의 결과를 포함하는 도구 메시지(Tool messages)**의 생성을 포함합니다.  
LangChain의 `tool_example_to_messages` 함수는 각 업체의 LLM에 맞추어 도구 호출 형식을 자동으로 포맷합니다. 

---

### **LangChain의 유틸리티 함수: `tool_example_to_messages`**  
각 모델 제공업체(OpenAI, Anthropic 등)마다 도구 호출 형식이 조금씩 다릅니다. LangChain의 tool_example_to_messages 함수는 Pydantic 객체만 있으면, 이를 모델 제공업체의 요구사항에 맞는 형식으로 자동 변환해주는 도구입니다. tool_example_to_messages는 개발자가 복잡한 포맷을 신경 쓰지 않고, Pydantic 객체만 제공하면 자동으로 올바른 메시지 형식을 만들어줍니다.

---

###  **동작 순서**  

1. 입력 문자열: 사용자의 요청이나 질문을 문자열로 제공합니다.  
2. Pydantic 객체: 사용할 도구나 함수의 입력 형식을 Pydantic 모델로 정의합니다.  
3. 자동 포맷: LangChain이 이 두 가지를 조합해 모델 제공업체(OpenAI, Anthropic 등)에 맞는 메시지 포맷으로 자동 변환합니다.


In [ ]:
# 도구 호출 예제를 메시지로 변환하기 위한 유틸리티 함수
# Few-Shot Prompting 예제 데이터 정의
# 입력 텍스트와 모델이 추출해야 할 데이터의 목표 형식(Target Schema) 쌍으로 정의
# 메시지 목록 초기화
# 예제 데이터를 메시지로 변환
    # tool_example_to_messages 함수를 사용해 입력 텍스트와 도구 호출 데이터를 메시지로 변환

결과를 살펴보면 다음 두 쌍의 예시가 8개의 메시지를 생성한 것을 알 수 있습니다.

----
이 메시지들을 Few-Shot Example로 Prompt에 포함할와 포함하지 않을 때의 성능을 비교해봅니다.   
다음은, 사람이 포함되지 않은 메시지에 대한 llm의 response 비교 입니다.

이 예제에서 모델은 사람에 대한 잘못된 응답을 생성할 가능성이 있습니다.  

그러나 "Detected no people" 사례가 포함되어 있는 **Few-Shot Example**을 제공할 경우, 모델이 올바르게 작동하도록 유도할 수 있습니다.